Updates to script for version 2

1.   Using PySpark & Spark Pandas API for better performance & speed
2.   Using Azure Cousmos DB to send cleaned data to PowerBI Dashboard
3.   Minor usage of Numba Framework to speed up some methods within this script
4.   Potential Trials of Time-Series Machine Learning Modeling of Data 
5.   Trials with using the Libra Library for potential simple Machine Learning development
6.   Determine if using Azure Synapse makes sense for this project for giving no-code manipulation of the data

Website Links & Documentation for all program enhancements

1.   PySpark Examples - https://github.com/apache/spark/tree/f74867bddf/examples/src/main/python
2.   Azure Cosmos DB - https://docs.microsoft.com/en-us/azure/cosmos-db/sql/create-sql-api-python
2.   Azure Cosmos DB Github Demo - https://github.com/Azure-Samples/azure-cosmos-db-python-getting-started/blob/main/cosmos_get_started.py
3.   Numba - https://numba.pydata.org/
4.   Time-Series Machine Learning Algos - https://www.advancinganalytics.co.uk/blog/2021/06/22/10-incredibly-useful-time-series-forecasting-algorithms
5.   Libra - https://libradocs.org/
6.   Azure Synapse & Cosmos DB Integration - https://docs.microsoft.com/en-us/azure/cosmos-db/synapse-link
7.   Spark SQL & Python - https://spark.apache.org/docs/latest/sql-programming-guide.html
8.   DeepHaven Streaming Dataframe - https://deephaven.io/?utm_term=streaming%20data&utm_campaign=Website+Traffic+Q4+2022&utm_source=adwords&utm_medium=ppc&hsa_acc=4673439537&hsa_cam=18322365595&hsa_grp=139855244374&hsa_ad=621564846172&hsa_src=g&hsa_tgt=kwd-161093182&hsa_kw=streaming%20data&hsa_mt=p&hsa_net=adwords&hsa_ver=3
9.   Spark SQL & Python - https://spark.apache.org/docs/latest/
10.  Google Sheets & Python - https://ploomber.io/blog/gsheets/

In [10]:
#all imports 
import gspread
import numpy as np
import pyspark.pandas as ps
import numba
from numba import jit
import os
import json
from datetime import *
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.errors as errors
import azure.cosmos.documents as documents
import azure.cosmos.http_constants as http_constants

In [22]:
#setup spark env & import spark variables
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [16]:
#call to the dataset within Google Sheets
sa = gspread.service_account(filename="/com.docker.devenvironments.code/kuthesisdataclean-558333f9362c.json")
sheet = sa.open("POU_Unclean_Data")
work_sheet = sheet.worksheet("POU")
rows = work_sheet.get_all_values()
#column_names = ['Item_no', 'Item Description', 'Quantity', 'Status', 'Serial Number', 'Reason Code', 'Shortage Notes', 'Team Found', 'Team Responsible', 'Data Found', 'Time Found Pacific', 'Data Found Pacific', 'SHORT_FOUND_EMP_ID', 'Date_chassis_start', 'date_veh_offln', 'date_mfg_rlse', 'PRIM_SHOP_ADDR', 'SCD_SHOP_ADDR_1', 'SCD_SHOP_ADDR_2', 'SCD_SHOP_ADDR_3']
#rows = rows.get_all_records()
df = ps.DataFrame(rows)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,Item_no,Item Description,Qty,Status,Serial Number,Reason Code,Shortage Notes,Team Found,Team Resp,Date Found,Time Found (Pacific),SHORT_FOUND_EMP_ID,Date_chassis_start,date_veh_offln,date_mfg_rlse,PRIM_SHOP_ADDR,SCD_SHOP_ADDR_1,SCD_SHOP_ADDR_2,SCD_SHOP_ADDR_3,SCD_SHOP_ADDR_4
1,03-33412-875,"ISOLATOR-FTK,BAND,M2,CBOC",2,T,UB2964,MHE,on kit cart ...,541,521,05/19/2022,05:02:55,ricrisp,2022-08-18,2022-08-18,,RPK 2,02817R,,,CRITIC
2,A14-16601-000,"STUBSHAFT-STEERING,M2,EPA07",1,I,NV7601,PLT,INSTALLED BUT NEEDS VERIFIED AND SIGNED OFF.....,207,207,07/25/2022,11:05:44,staffod,2022-08-11,2022-08-12,2022-08-14,TUG03,61113R,,,CRITIC
3,03-40538-002,"FWS-DETROIT,WIF",1,I,NV7659,VPD,TIED TO MOUNTING BRKT.. S WELCH ...,501,501,07/25/2022,08:23:47,MILLETA,2022-08-11,2022-08-12,2022-08-19,60516E,02524R,02524R,,
4,04-16509-010,"PIPE-5""X10""PLN,CRV",1,I,UD5183,MID,production wrote up part and did not nee d the...,249,926,07/28/2022,19:49:52,MASBURY,2022-08-15,2022-08-16,,6600C,02700,,,


In [27]:
spark_df = spark.createDataFrame(rows)
spark_df = spark.read.format('csv').option('header', True)
spark_df.head()

AttributeError: 'DataFrameReader' object has no attribute 'head'

In [17]:
#Next we are going to format the rows to allow for the data to get into a data frame
df.columns = df.iloc[0] 
df = df[1:]
df.head()

,Item_no,Item Description,Qty,Status,Serial Number,Reason Code,Shortage Notes,Team Found,Team Resp,Date Found,Time Found (Pacific),SHORT_FOUND_EMP_ID,Date_chassis_start,date_veh_offln,date_mfg_rlse,PRIM_SHOP_ADDR,SCD_SHOP_ADDR_1,SCD_SHOP_ADDR_2,SCD_SHOP_ADDR_3,SCD_SHOP_ADDR_4
1,03-33412-875,"ISOLATOR-FTK,BAND,M2,CBOC",2,T,UB2964,MHE,on kit cart ...,541,521,05/19/2022,05:02:55,ricrisp,2022-08-18,2022-08-18,,RPK 2,02817R,,,CRITIC
2,A14-16601-000,"STUBSHAFT-STEERING,M2,EPA07",1,I,NV7601,PLT,INSTALLED BUT NEEDS VERIFIED AND SIGNED OFF.....,207,207,07/25/2022,11:05:44,staffod,2022-08-11,2022-08-12,2022-08-14,TUG03,61113R,,,CRITIC
3,03-40538-002,"FWS-DETROIT,WIF",1,I,NV7659,VPD,TIED TO MOUNTING BRKT.. S WELCH ...,501,501,07/25/2022,08:23:47,MILLETA,2022-08-11,2022-08-12,2022-08-19,60516E,02524R,02524R,,
4,04-16509-010,"PIPE-5""X10""PLN,CRV",1,I,UD5183,MID,production wrote up part and did not nee d the...,249,926,07/28/2022,19:49:52,MASBURY,2022-08-15,2022-08-16,,6600C,02700,,,
5,A17-21151-028,"HOOD-106,BZL",1,Y,UK8929,VPD,VENDOR SHORT ...,,,07/29/2022,10:25:19,,2022-08-18,,,32500,,,,


In [ ]:
# Initialize the Cosmos client
config = {
    "endpoint": "https://ketteringthesis.documents.azure.com:443/",
    "primarykey": "FDY0Eiwlwkm9bFUxRVLPDot54mcVvIoVCjH0gV3AL3YxUzigmhuQhMGonDV01S0FKH1MqJJ78wOGdrQGHqx2cA=="
}

# Create the cosmos client
client = cosmos_client.CosmosClient(url_connection=config["endpoint"], auth={"masterKey":config["primarykey"]}
)